In [1]:
from src.helper_functions import hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.symmetric_encryption import is_ecb_mode
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 17: PKCS#7 padding validation**

In [2]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_17_key = secrets.token_bytes(16)

In [3]:
def encryption_oracle_17() -> tuple[bytes, bytes]:
    plaintext_list = [
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93",
    ]

    chosen_plaintext = secrets.choice(plaintext_list)
    chosen_plaintext = base64.b64decode(chosen_plaintext)
    chosen_plaintext = padding.apply_pkcs_7(chosen_plaintext, aes128.AES128_KEY_LENGTH)
    
    IV = secrets.token_bytes(16)
    return aes128.cbc_encrypt(chosen_plaintext, encryption_oracle_17_key, IV), IV


def correct_padding_oracle(ciphertext: bytes, IV: bytes) -> bool:
    plaintext = aes128.cbc_decrypt(ciphertext, encryption_oracle_17_key, IV)
    try:
        padding.remove_pkcs_7(plaintext)
        return True
    except Exception:
        return False

In [4]:
cyphertext, IV = encryption_oracle_17()
correct_padding_oracle(cyphertext, IV)

True

In [5]:
cyphertext_blocks = [IV] + convert.bytes2blocks(cyphertext, aes128.AES128_KEY_LENGTH)
cyphertext_blocks

[b'\x1f&j\x00\xd3\xa0\xd2\xf9s\x89\xe1\x0c\x05\xd9\xa7U',
 b'\x9cE\x8c\xc2N$\xc4\xd0\n\x8f\xb7\xec\x1f\xf4C\xf6',
 b'6\xc9\xaa\x97is\xc1\xdc\xbf\x05f\xe8\xdfX\xca\xf2',
 b'\x8a\xbd\x9e\xa2xL\xf0\xec\xaemq\\q\xbd6\xcd']

In [6]:

# # 1 * convert.int2byte(1)
# plaintext_block = aes128.AES128_KEY_LENGTH * convert.int2byte(0)

# zeros_IV = aes128.AES128_KEY_LENGTH * convert.int2byte(0)

# for i in range(256):
#     plaintext_block = 15 * convert.int2byte(0) + convert.int2byte(i)
#     padding_block = padding.apply_pkcs_7(convert.int2byte(0) * (aes128.AES128_KEY_LENGTH - 1), aes128.AES128_KEY_LENGTH)
#     fake_IV = xor_bytes(plaintext_block, xor_bytes(padding_block, cyphertext_blocks[1]))

#     zeroing_IV = xor_bytes(padding_block, cyphertext_blocks[1])

#     if correct_padding_oracle(cyphertext_blocks[1], fake_IV):
#         print(i)





## **Challenge 18: Implement CTR, the stream cipher mode**

In [7]:
cyphertext = base64.b64decode("L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==")
key = b"YELLOW SUBMARINE"
plaintext = aes128.ctr_decrypt(cyphertext, key, 0)
print(plaintext.decode())

Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


## **Challenge 19: Break fixed-nonce CTR mode using substitutions**

In [8]:
encryption_oracle_19_key = secrets.token_bytes(16)

In [9]:
plaintext_list = [
    "SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==",
    "Q29taW5nIHdpdGggdml2aWQgZmFjZXM=",
    "RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==",
    "RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=",
    "SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk",
    "T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
    "T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=",
    "UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
    "QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=",
    "T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl",
    "VG8gcGxlYXNlIGEgY29tcGFuaW9u",
    "QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==",
    "QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=",
    "QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==",
    "QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=",
    "QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=",
    "VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==",
    "SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==",
    "SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==",
    "VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==",
    "V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==",
    "V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==",
    "U2hlIHJvZGUgdG8gaGFycmllcnM/",
    "VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=",
    "QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=",
    "VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=",
    "V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=",
    "SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==",
    "U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==",
    "U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=",
    "VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==",
    "QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu",
    "SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=",
    "VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs",
    "WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=",
    "SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0",
    "SW4gdGhlIGNhc3VhbCBjb21lZHk7",
    "SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=",
    "VHJhbnNmb3JtZWQgdXR0ZXJseTo=",
    "QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=",
]
plaintext_list = [base64.b64decode(plaintext) for plaintext in plaintext_list]

In [10]:
for plaintext in plaintext_list:
    print(aes128.ctr_decrypt(plaintext, encryption_oracle_19_key, 0))

b'\xb9C\n7F\xd6\x00!\xb8\x0c/\xf5HK\xba\x1a\xf8\xc2B>=\xdd\xf8\x08W/@\x97\xcaH\x93'
b'\xb3\x0c\x0f?^\xd4\x00;\xb4\x0cg\xa1VG\xa1S\xfd\x96\x04<2\xd7\xf8'
b"\xb6\x11\r;\x10\xd0O9\xb3\x0cj\xf3\x00A\xa5\x1a\xfd\xd3\x116q\xd3\xe6\x02\x19'\x06\xd0\xdcL\x93"
b'\xb5\n\x05>D\xd6E"\xa9\x10"\xe2E@\xa3O\xeb\xcfB5>\xc7\xf8\x08\x04n'
b'\xb9C\n7F\xd6\x00<\xbc\x0b|\xe4D\x0e\xa0S\xed\xdeB<q\xdc\xe4\tW/@\x97\xdaA\x8fi\x82}p\x9b'
b'\xbf\x11B&_\xdfI8\xb8Xb\xe4A@\xbeT\xfe\xda\x07."\x92\xfc\x02\x05$U\x9b'
b'\xbf\x11B>Q\xc5El\xb1\x11a\xe6E\\\xb2^\xb9\xd7\x1558\xde\xeeM\x16.B\x97\xddH\x83-'
b'\xa0\x0c\x0e?D\xd6\x00!\xb8\x19a\xe8NI\xbb_\xea\xc5B*>\xc0\xef\x1e['
b'\xb1\r\x06vD\xdbO9\xba\x10{\xa1BK\xb1U\xeb\xd3B\x14q\xda\xea\tW$I\xd9\xcb'
b'\xbf\x05B7\x10\xdeO/\xb6\x11a\xe6\x00Z\xb6V\xfc\x96\r/q\xd3\xab\n\x1e"C'
b'\xa4\x0cB&\\\xd6A?\xb8Xn\xa1CA\xbaJ\xf8\xd8\x0b2?'
b'\xb1\x11\r#^\xd7\x008\xb5\x1d/\xe7I\\\xb2\x1a\xf8\xc2B)9\xd7\xab\x0e\x1b5D\x9b'
b'\xb2\x06\x0b8W\x93C)\xaf\x0cn\xe8N\x0e\xa3R\xf8\xc2B)9\xd7\xf2M\x1